In [0]:
# set to True if running in Google Colaboratory else False
colab = True

In [2]:
### import packages
if colab:
    try:
        # %tensorflow_version only exists in Colab.
        %tensorflow_version 2.x
    except Exception:
        pass

import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

TensorFlow 2.x selected.


In [3]:
### clone GitHub repository and unzip data if running in Google Colab
if colab:
    !git clone https://github.com/atotschnig/fashion-MNIST.git
    %cd fashion-MNIST/
    !unzip data.zip
    !rm -rf __MACOSX

Cloning into 'fashion-MNIST'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 14 (delta 2), reused 11 (delta 2), pack-reused 3
Unpacking objects: 100% (14/14), done.
/content/fashion-MNIST
Archive:  data.zip
   creating: data/
  inflating: data/train_labels.csv   
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._train_labels.csv  
  inflating: data/label_int_to_str_mapping.csv  
  inflating: __MACOSX/data/._label_int_to_str_mapping.csv  
  inflating: data/train_images.npy   
  inflating: __MACOSX/data/._train_images.npy  
  inflating: data/test_images.npy    
  inflating: __MACOSX/data/._test_images.npy  
  inflating: __MACOSX/._data         


In [0]:
# set path variables
path_data = 'data'
path_submission = 'submissions'

In [0]:
### helper functions
def process_images(X, augment=False):

    # normalize image data
    X = X / 255

    # reshape into (n_samples, width, height, 1)
    X = X.reshape(X.shape + (1,))

    if augment:
        # vertical reflection to augment data set
        X_vertical = np.flip(X,2)
        X = np.concatenate((X, X_vertical), 0)
    return X

def process_labels(y, augment=False):
    
    # drop 'ID' column
    if augment:
        return  np.concatenate((y[:,1], y[:,1]), 0) 

    else:
        return y[:,1]

def predict(X, model):
    y = model.predict(X)
    return np.argmax(y, axis=1)

def categorical_to_label(y):
    return np.argmax(y, axis=1)

def get_acc(X, y, model):
    preds = predict(X, model)
    # acc = np.mean(preds==categorical_to_label(y))
    acc = np.mean(preds==y)
    return acc

In [0]:
def get_model_1():
    """model architecture based on Kaggle kernel https://www.kaggle.com/bugraokcu/cnn-with-keras"""

    model = Sequential([
        Conv2D(filters=32, kernel_size=3, activation='relu', kernel_initializer='he_normal', input_shape=im_shape),
        MaxPooling2D(2,2),
        Dropout(0.25),
        Conv2D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.25),
        Conv2D(filters=128, kernel_size=3, activation='relu'),
        Dropout(0.4),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer=Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  
    return model

In [0]:
def get_model_2():
    """madel architecture based on Kaggle kernel https://www.kaggle.com/fuzzywizard/fashion-mnist-cnn-keras-accuracy-93"""

    model = Sequential([
        Conv2D(filters=32, kernel_size=3, activation='relu', kernel_initializer='he_normal', input_shape=im_shape),
        BatchNormalization(),
        Conv2D(filters=32, kernel_size=3, activation='relu'),
        BatchNormalization(),
        Dropout(0.25),
        Conv2D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.25),
        Conv2D(filters=128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        Dropout(0.25),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer=Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  
    return model

In [0]:
def get_model_3():
    """modified version of model 2"""

    model = Sequential([
        Conv2D(filters=32, kernel_size=3, activation='relu', kernel_initializer='he_normal', input_shape=im_shape),
        BatchNormalization(),
        Conv2D(filters=32, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        Conv2D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.25),
        Conv2D(filters=128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer=Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  
    return model

In [0]:
### code for k-fold cross-validation (not used)

# epochs = 25
# batch_size = 256
# n_splits=10

# # load images and labels
# X = np.load('train_images.npy')
# y = np.loadtxt('train_labels.csv', delimiter=',', skiprows=1)

# # process data
# X = process_images(X)
# y = process_labels(y)

# # get image shape
# im_shape = X.shape[1:]

# kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=3791)

# acc = []
# val_acc = []

# for train, val in kfold.split(X, y):

#   model = get_model()
#   model.fit(X[train], y[train], epochs=epochs, batch_size=batch_size, validation_data=(X[val], y[val]))

#   acc.append(get_acc(X[train], y[train], model))
#   val_acc.append(get_acc(X[val], y[val], model))

# print('Mean training accuracy:', np.mean(acc))
# print('Mean validation accuracy:', np.mean(val_acc))

In [10]:
### train model

models = {1:get_model_1, 2:get_model_2, 3:get_model_3}

# hyperparameters for final model
model_id = 2
augment = True
epochs = 25
batch_size = 32

# load images and labels
X_train = np.load(os.path.join(path_data, 'train_images.npy'))
y_train = np.loadtxt(os.path.join(path_data, 'train_labels.csv'), delimiter=',', skiprows=1)

# process data
X_train = process_images(X_train, augment=augment)
y_train = process_labels(y_train, augment=augment)

# get image shape
im_shape = X_train.shape[1:]

# train-validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=3791)

# model = get_model_1()
model = models[model_id]()
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val,y_val))
print('Training accuracy:', get_acc(X_train, y_train, model))
print('Validation accuracy:', get_acc(X_val, y_val, model))

Train on 90000 samples, validate on 10000 samples
Epoch 1/25
90000/90000 [==============================] - 53s 589us/sample - loss: 0.6672 - accuracy: 0.7615 - val_loss: 0.4379 - val_accuracy: 0.8345
Epoch 2/25
90000/90000 [==============================] - 47s 526us/sample - loss: 0.4694 - accuracy: 0.8308 - val_loss: 0.4196 - val_accuracy: 0.8496
Epoch 3/25
90000/90000 [==============================] - 47s 527us/sample - loss: 0.4184 - accuracy: 0.8504 - val_loss: 0.3512 - val_accuracy: 0.8642
Epoch 4/25
90000/90000 [==============================] - 48s 528us/sample - loss: 0.3789 - accuracy: 0.8627 - val_loss: 0.3151 - val_accuracy: 0.8816
Epoch 5/25
90000/90000 [==============================] - 48s 528us/sample - loss: 0.3554 - accuracy: 0.8713 - val_loss: 0.3023 - val_accuracy: 0.8856
Epoch 6/25
90000/90000 [==============================] - 48s 532us/sample - loss: 0.3322 - accuracy: 0.8803 - val_loss: 0.2836 - val_accuracy: 0.8948
Epoch 7/25
90000/90000 [====================

In [0]:
### create submission.csv file

# test model
X_test = np.load(os.path.join(path_data, 'test_images.npy')).squeeze()
y_test = predict(process_images(X_test), model)

# save predictions
df_test = pd.read_csv(os.path.join(path_submission, 'sample_submission.csv'))
df_test['label'] = y_test
df_test.to_csv(os.path.join(path_submission, 'submission.csv'), index=False)